# dice (Dice)

In [ ]:
//// test

open testing

## sixth_power_sequence

In [ ]:
inl sixth_power_sequence () =
    stream.iterate ((*) 6) 1

In [ ]:
//// test

sixth_power_sequence ()
|> stream.take_while (fun _ i => i <= 7i32)
|> stream.to_list
|> _assert_eq [ 1i32; 6; 36; 216; 1296; 7776; 46656; 279936 ]

__assert_eq / actual: UH0_1
  (1,
   UH0_1
     (6,
      UH0_1
        (36,
         UH0_1
           (216,
            UH0_1 (1296, UH0_1 (7776, UH0_1 (46656, UH0_1 (279936, UH0_0)))))))) / expected: UH0_1
  (1,
   UH0_1
     (6,
      UH0_1
        (36,
         UH0_1
           (216,
            UH0_1 (1296, UH0_1 (7776, UH0_1 (46656, UH0_1 (279936, UH0_0))))))))


## accumulate_dice_rolls

In [ ]:
inl accumulate_dice_rolls
    (log : option (string -> ()))
    (rolls : list u8)
    (power : i8)
    acc
    =
    inl rec body acc power rolls =
        match rolls with
        | _ when power < 0 =>
            inl result = acc + 1
            log |> optionm'.iter ((|>) $'$"accumulate_dice_rolls / power: {!power} / acc: {!acc} / result: {!result}"')
            Some (result, rolls)
        | [] => None
        | roll :: rest when roll > 1 =>
            inl coeff = sixth_power_sequence () |> stream.item power
            inl value = conv (roll - 1) * coeff
            log |> optionm'.iter ((|>) $'$"accumulate_dice_rolls / power: {!power} / acc: {!acc} / roll: {!roll} / value: {!value}"'
            )
            loop (acc + value) (power - 1) rest
        | roll :: rest =>
            log |> optionm'.iter ((|>) $'$"accumulate_dice_rolls / power: {!power} / acc: {!acc} / roll: {!roll}"')
            loop acc (power - 1) rest
    and inl loop acc power rolls =
        if var_is rolls |> not
        then body acc power rolls
        else
            inl acc = dyn acc
            join body acc power rolls
    loop acc power rolls

In [ ]:
//// test

accumulate_dice_rolls (Some console.write_line) [ 6; 5; 4; 3; 2 ] 0 1000i32
|> _assert_eq (Some (1006, [ 5; 4; 3; 2 ]))

accumulate_dice_rolls / power: 0 / acc: 1000 / roll: 6 / value: 5
accumulate_dice_rolls / power: -1 / acc: 1005 / result: 1006
__assert_eq / actual: US0_0 (1006, UH0_1 (5uy, UH0_1 (4uy, UH0_1 (3uy, UH0_1 (2uy, UH0_0))))) / expected: US0_0 (1006, UH0_1 (5uy, UH0_1 (4uy, UH0_1 (3uy, UH0_1 (2uy, UH0_0)))))


In [ ]:
//// test

accumulate_dice_rolls (Some console.write_line) [ 6; 5; 4; 3; 2 ] 1 1000i32
|> _assert_eq (Some (1035, [ 4; 3; 2 ]))

accumulate_dice_rolls / power: 1 / acc: 1000 / roll: 6 / value: 30
accumulate_dice_rolls / power: 0 / acc: 1030 / roll: 5 / value: 4
accumulate_dice_rolls / power: -1 / acc: 1034 / result: 1035
__assert_eq / actual: US0_0 (1035, UH0_1 (4uy, UH0_1 (3uy, UH0_1 (2uy, UH0_0)))) / expected: US0_0 (1035, UH0_1 (4uy, UH0_1 (3uy, UH0_1 (2uy, UH0_0))))


In [ ]:
//// test

accumulate_dice_rolls (Some console.write_line) [ 6; 5; 4; 3; 2 ] 2 1000i32
|> _assert_eq (Some (1208, [ 3; 2 ]))

accumulate_dice_rolls / power: 2 / acc: 1000 / roll: 6 / value: 180
accumulate_dice_rolls / power: 1 / acc: 1180 / roll: 5 / value: 24
accumulate_dice_rolls / power: 0 / acc: 1204 / roll: 4 / value: 3
accumulate_dice_rolls / power: -1 / acc: 1207 / result: 1208
__assert_eq / actual: US0_0 (1208, UH0_1 (3uy, UH0_1 (2uy, UH0_0))) / expected: US0_0 (1208, UH0_1 (3uy, UH0_1 (2uy, UH0_0)))


## roll_within_bounds

In [ ]:
inl roll_within_bounds (log : option (string -> ())) max rolls =
    inl power = listm.length rolls - 1
    match accumulate_dice_rolls log rolls power 0 with
    | Some (result, _) when result >= 1 && result <= max => Some result
    | _ => None

In [ ]:
//// test

roll_within_bounds (Some console.write_line) 2000i32 [ 1; 5; 4; 4; 5 ]
|> _assert_eq (Some 995)

accumulate_dice_rolls / power: 4 / acc: 0 / roll: 1
accumulate_dice_rolls / power: 3 / acc: 0 / roll: 5 / value: 864
accumulate_dice_rolls / power: 2 / acc: 864 / roll: 4 / value: 108
accumulate_dice_rolls / power: 1 / acc: 972 / roll: 4 / value: 18
accumulate_dice_rolls / power: 0 / acc: 990 / roll: 5 / value: 4
accumulate_dice_rolls / power: -1 / acc: 994 / result: 995
__assert_eq / actual: US0_0 995 / expected: US0_0 995


In [ ]:
//// test

roll_within_bounds (Some console.write_line) 2000i32 [ 2; 2; 6; 4; 5 ]
|> _assert_eq (Some 1715)

accumulate_dice_rolls / power: 4 / acc: 0 / roll: 2 / value: 1296
accumulate_dice_rolls / power: 3 / acc: 1296 / roll: 2 / value: 216
accumulate_dice_rolls / power: 2 / acc: 1512 / roll: 6 / value: 180
accumulate_dice_rolls / power: 1 / acc: 1692 / roll: 4 / value: 18
accumulate_dice_rolls / power: 0 / acc: 1710 / roll: 5 / value: 4
accumulate_dice_rolls / power: -1 / acc: 1714 / result: 1715
__assert_eq / actual: US0_0 1715 / expected: US0_0 1715


In [ ]:
//// test

roll_within_bounds (Some console.write_line) 2000i32 [ 4; 1; 1; 2; 3 ]
|> _assert_eq None

accumulate_dice_rolls / power: 4 / acc: 0 / roll: 4 / value: 3888
accumulate_dice_rolls / power: 3 / acc: 3888 / roll: 1
accumulate_dice_rolls / power: 2 / acc: 3888 / roll: 1
accumulate_dice_rolls / power: 1 / acc: 3888 / roll: 2 / value: 6
accumulate_dice_rolls / power: 0 / acc: 3894 / roll: 3 / value: 2
accumulate_dice_rolls / power: -1 / acc: 3896 / result: 3897
__assert_eq / actual: US0_1 / expected: US0_1


## calculate_dice_count

In [ ]:
inl calculate_dice_count (log : option (string -> ())) max =
    inl rec body n p =
        inl return () =
            log |> optionm'.iter ((|>) $'$"calculate_dice_count / max: {!max} / n: {!n} / p: {!p}"')
            n

        if p < max then
            inl p' = p * 6
            if p' > p
            then loop (n + 1) p'
            else return ()
        else return ()

    and inl loop n p =
        if var_is max |> not
        then body n p
        else
            inl n = dyn n
            inl p = dyn p
            join body n p
    if max = 1
    then 1
    else loop 0 1

In [ ]:
//// test

calculate_dice_count (Some console.write_line) 36i32
|> _assert_eq 2i32

calculate_dice_count / max: 36 / n: 2 / p: 36
__assert_eq / actual: 2 / expected: 2


In [ ]:
//// test

calculate_dice_count (Some console.write_line) 7777i32
|> _assert_eq 6i32

calculate_dice_count / max: 7777 / n: 6 / p: 46656
__assert_eq / actual: 6 / expected: 6


## roll_dice

In [ ]:
let roll_dice () : u8 =
    random' 1 7

## rotate_number

In [ ]:
inl rotate_number forall t {number}. (max : i64) (n : t) : t =
    (conv n - 1 + max) % max + 1 |> conv

## rotate_numbers

In [ ]:
inl rotate_numbers max items =
    items |> stream.map (rotate_number max)

In [ ]:
//// test

listm'.init_series -1 14 1i32
|> stream.from_list
|> rotate_numbers 6
|> stream.to_list
|> _assert_eq [ 5; 6; 1; 2; 3; 4; 5; 6; 1; 2; 3; 4; 5; 6; 1; 2 ]

__assert_eq / actual: UH0_1
  (5,
   UH0_1
     (6,
      UH0_1
        (1,
         UH0_1
           (2,
            UH0_1
              (3,
               UH0_1
                 (4,
                  UH0_1
                    (5,
                     UH0_1
                       (6,
                        UH0_1
                          (1,
                           UH0_1
                             (2,
                              UH0_1
                                (3,
                                 UH0_1
                                   (4,
                                    UH0_1
                                      (5, UH0_1 (6, UH0_1 (1, UH0_1 (2, UH0_0)))))))))))))))) / expected: UH0_1
  (5,
   UH0_1
     (6,
      UH0_1
        (1,
         UH0_1
           (2,
            UH0_1
              (3,
               UH0_1
                 (4,
                  UH0_1
                    (5,
                     UH0_1
                       (6,
                        UH

## create_sequential_roller

In [ ]:
inl create_sequential_roller (log : option (string -> ())) list =
    log |> optionm'.iter ((|>) $'$"create_sequential_roller ()"')
    inl s = list |> listm.rev |> listm.append list |> stream.from_list |> stream.memoize
    inl current_index = mut 0i64
    inl acc = mut 1
    inl len = mut -1
    inl last_item = mut None
    let rec loop () =
        fun () =>
            inl current_index = *current_index
            inl acc = *acc
            inl len = *len
            inl last_item = *last_item
            log |> optionm'.iter ((|>) $'$"create_sequential_roller / roll / current_index: {!current_index} / acc: {!acc} / len: {!len} / last_item: %A{!last_item}"')
        |> fun x => x ()
        match s () |> stream.try_item *current_index with
        | Some item =>
            current_index <- *current_index + 1
            last_item <- Some item
            item
        | None =>
            log |> optionm'.iter ((|>) $'$"create_sequential_roller / roll / None"')
            if *len = -1
            then len <- *current_index
            acc <-
                if *acc >= *len
                then 1
                else *acc + 1
            current_index <- *acc - 1
            last_item <- None
            loop ()
    loop

In [ ]:
//// test

inl sequential_roll = create_sequential_roller (Some console.write_line) [ 1i32; 2; 3; 4 ]

am.init 50i32 (ignore >> sequential_roll)
|> _assert_eq (a ;[ 1; 2; 3; 4; 4; 3; 2; 1; 2; 3; 4; 4; 3; 2; 1; 3; 4; 4; 3; 2; 1; 4; 4; 3; 2; 1;
4; 3; 2; 1; 3; 2; 1; 2; 1; 1; 1; 2; 3; 4; 4; 3; 2; 1; 2; 3; 4; 4; 3; 2 ] : a i32 i32)

create_sequential_roller ()
create_sequential_roller / roll / current_index: 0 / acc: 1 / len: -1 / last_item: US1_1
create_sequential_roller / roll / current_index: 1 / acc: 1 / len: -1 / last_item: US1_0 1
create_sequential_roller / roll / current_index: 2 / acc: 1 / len: -1 / last_item: US1_0 2
create_sequential_roller / roll / current_index: 3 / acc: 1 / len: -1 / last_item: US1_0 3
create_sequential_roller / roll / current_index: 4 / acc: 1 / len: -1 / last_item: US1_0 4
create_sequential_roller / roll / current_index: 5 / acc: 1 / len: -1 / last_item: US1_0 4
create_sequential_roller / roll / current_index: 6 / acc: 1 / len: -1 / last_item: US1_0 3
create_sequential_roller / roll / current_index: 7 / acc: 1 / len: -1 / last_item: US1_0 2
create_sequential_roller / roll / current_index: 8 / acc: 1 / len: -1 / last_item: US1_0 1
create_sequential_roller / roll / None
create_sequential_roller / roll / current_index: 1 / acc: 2 / len: 8 / last_item: US1_1
create_sequential_roller / r

## roll_progressively

In [ ]:
inl roll_progressively (log : option (string -> ())) roll reroll max =
    inl power = (calculate_dice_count log max) - 1
    let rec loop rolls size =
        if size < power + 1
        then loop (roll () :: rolls) (size + 1)
        else accumulate_dice_rolls log rolls power 0 |> function
            | Some (result, _) when result <= max => result
            | _ when reroll => loop (listm.init power (fun _ => roll ())) power
            | _ => loop (roll () :: rolls) (size + 1)
    loop [] 0

In [ ]:
//// test

roll_progressively None roll_dice false 1
|> _assert_eq 1i32

__assert_eq / actual: 1 / expected: 1


In [ ]:
//// test

inl sequential_roll = create_sequential_roller (Some console.write_line) [ 5; 4; 4; 5; 1 ]

roll_progressively (Some console.write_line) sequential_roll false 2000i32
|> _assert_eq 995

create_sequential_roller ()
calculate_dice_count / max: 2000 / n: 5 / p: 7776
create_sequential_roller / roll / current_index: 0 / acc: 1 / len: -1 / last_item: US1_1
create_sequential_roller / roll / current_index: 1 / acc: 1 / len: -1 / last_item: US1_0 5uy
create_sequential_roller / roll / current_index: 2 / acc: 1 / len: -1 / last_item: US1_0 4uy
create_sequential_roller / roll / current_index: 3 / acc: 1 / len: -1 / last_item: US1_0 4uy
create_sequential_roller / roll / current_index: 4 / acc: 1 / len: -1 / last_item: US1_0 5uy
accumulate_dice_rolls / power: 4 / acc: 0 / roll: 1
accumulate_dice_rolls / power: 3 / acc: 0 / roll: 5 / value: 864
accumulate_dice_rolls / power: 2 / acc: 864 / roll: 4 / value: 108
accumulate_dice_rolls / power: 1 / acc: 972 / roll: 4 / value: 18
accumulate_dice_rolls / power: 0 / acc: 990 / roll: 5 / value: 4
accumulate_dice_rolls / power: -1 / acc: 994 / result: 995
__assert_eq / actual: 995 / expected: 995


In [ ]:
//// test

inl sequential_roll = create_sequential_roller (Some console.write_line) [ 5; 4; 4; 5; 2 ]

roll_progressively (Some console.write_line) sequential_roll false 2000i32
|> _assert_eq 1678

create_sequential_roller ()
calculate_dice_count / max: 2000 / n: 5 / p: 7776
create_sequential_roller / roll / current_index: 0 / acc: 1 / len: -1 / last_item: US1_1
create_sequential_roller / roll / current_index: 1 / acc: 1 / len: -1 / last_item: US1_0 5uy
create_sequential_roller / roll / current_index: 2 / acc: 1 / len: -1 / last_item: US1_0 4uy
create_sequential_roller / roll / current_index: 3 / acc: 1 / len: -1 / last_item: US1_0 4uy
create_sequential_roller / roll / current_index: 4 / acc: 1 / len: -1 / last_item: US1_0 5uy
accumulate_dice_rolls / power: 4 / acc: 0 / roll: 2 / value: 1296
accumulate_dice_rolls / power: 3 / acc: 1296 / roll: 5 / value: 864
accumulate_dice_rolls / power: 2 / acc: 2160 / roll: 4 / value: 108
accumulate_dice_rolls / power: 1 / acc: 2268 / roll: 4 / value: 18
accumulate_dice_rolls / power: 0 / acc: 2286 / roll: 5 / value: 4
accumulate_dice_rolls / power: -1 / acc: 2290 / result: 2291
create_sequential_roller / roll / current_index: 5 / acc: 1 / len

In [ ]:
//// test

join dyn listm'.init_series 1i32 10000 1
|> listm.map (fun _ => roll_progressively None roll_dice false 10i32)
|> listm'.group_by id
|> listm.map (fun (k, v) => k, (listm.length v : i32))
|> listm'.box
|> listm'.to_array'
|> fun ar =>
    $'!ar |> Array.sortBy (fun (struct (a, b)) -> a)' : a i32 $'struct (int32 * int32)'

index value 0 (1, 289) Item1 1 Item2 289 1 (2, 728) Item1 2 Item2 728 2 (3, 1385) Item1 3 Item2 1385 3 (4, 1283) Item1 4 Item2 1283 4 (5, 1365) Item1 5 Item2 1365 5 (6, 1316) Item1 6 Item2 1316 6 (7, 285) Item1 7 Item2 285 7 (8, 716) Item1 8 Item2 716 8 (9, 1332) Item1 9 Item2 1332 9 (10, 1301) Item1 10 Item2 1301

In [ ]:
//// test

join dyn listm'.init_series 1i32 10000 1
|> listm.map (fun _ => roll_progressively None roll_dice true 10i32)
|> listm'.group_by id
|> listm.map (fun (k, v) => k, (listm.length v : i32))
|> listm'.box
|> listm'.to_array'
|> fun ar =>
    $'!ar |> Array.sortBy (fun (struct (a, b)) -> a)' : a i32 $'struct (int32 * int32)'

index value 0 (1, 1000) Item1 1 Item2 1000 1 (2, 1004) Item1 2 Item2 1004 2 (3, 1013) Item1 3 Item2 1013 3 (4, 1026) Item1 4 Item2 1026 4 (5, 973) Item1 5 Item2 973 5 (6, 1021) Item1 6 Item2 1021 6 (7, 1009) Item1 7 Item2 1009 7 (8, 996) Item1 8 Item2 996 8 (9, 959) Item1 9 Item2 959 9 (10, 999) Item1 10 Item2 999

In [ ]:
//// test
//// timeout=180000

join dyn listm'.init_series 1i32 100 1
|> listm.iter fun n =>
    listm'.init_series 0i32 1 1
    |> listm.iter fun reroll =>
        join dyn listm'.init_series 1i32 3500 1
        |> listm.map fun _ => roll_progressively None roll_dice (reroll = 1) n
        |> listm'.group_by id
        |> listm.length
        |> _assert_eq n

__assert_eq / actual: 1 / expected: 1
__assert_eq / actual: 1 / expected: 1
__assert_eq / actual: 2 / expected: 2
__assert_eq / actual: 2 / expected: 2
__assert_eq / actual: 3 / expected: 3
__assert_eq / actual: 3 / expected: 3
__assert_eq / actual: 4 / expected: 4
__assert_eq / actual: 4 / expected: 4
__assert_eq / actual: 5 / expected: 5
__assert_eq / actual: 5 / expected: 5
__assert_eq / actual: 6 / expected: 6
__assert_eq / actual: 6 / expected: 6
__assert_eq / actual: 7 / expected: 7
__assert_eq / actual: 7 / expected: 7
__assert_eq / actual: 8 / expected: 8
__assert_eq / actual: 8 / expected: 8
__assert_eq / actual: 9 / expected: 9
__assert_eq / actual: 9 / expected: 9
__assert_eq / actual: 10 / expected: 10
__assert_eq / actual: 10 / expected: 10
__assert_eq / actual: 11 / expected: 11
__assert_eq / actual: 11 / expected: 11
__assert_eq / actual: 12 / expected: 12
__assert_eq / actual: 12 / expected: 12
__assert_eq / actual: 13 / expected: 13
__assert_eq / actual: 13 / expected:

## main

In [ ]:
inl main (_args : array_base string) =
    inl result = roll_progressively (Some console.write_line) roll_dice true (limit.max : i64)
    console.write_line ($'$"result: {!result}"' : string)
    0i32

inl main () =
    inl rotate_numbers' : i64 -> stream.stream u8 -> stream.stream u8 =
        rotate_numbers

    inl create_sequential_roller' :
        optionm'.option' (string -> ())
        -> list u8 
        -> (() -> u8)
        =
        optionm'.unbox >> create_sequential_roller

    inl roll_progressively' :
        optionm'.option' (string -> ()) 
        -> (() -> u8)
        -> bool
        -> u64
        -> u64
        =
        optionm'.unbox >> roll_progressively

    inl roll_within_bounds' :
        optionm'.option' (string -> ())
        -> u64
        -> list u8
        -> optionm'.option' u64
        =
        fun a b c =>
            (optionm'.unbox >> roll_within_bounds) a b c
            |> optionm'.box

    $'let rotate_numbers x = !rotate_numbers' x' : ()
    $'let create_sequential_roller x = !create_sequential_roller' x' : ()
    $'let roll_progressively x = !roll_progressively' x' : ()
    $'let roll_within_bounds x = !roll_within_bounds' x' : ()
    $'let main args = !main args' : ()